In [3]:
# Importing Libraries
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError


In [42]:
def crawler():
    """This function crawls nepstock website and returns some data.

    Returns:
        [type]: [description]
    """
    try:
        html = urlopen('http://www.nepalstock.com')
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('The server could not be found!')
    else:
        print('Website Successfully Crawled.')
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    index = bs.find('div', {'class':{'current-index'}}).text.split()[0]
    market_status = bs.find('div',{'id':'market_info'}).find('div',{'class':'top_marketinfo'}).text.strip()
    market_info = [{data.text.split('|')[0]:data.text.split('|')[1]} for data in bs.find('div',{'id':'market_info'}).find_all('span')]

    tables = bs.find('div', {'id':{'nepse-stats'}}).find_all('table', {'class':{'table table-hover table-condensed'}})

    alldata=[]
    for table in tables:
        t={}
        for row in table.find("thead").findAll('tr'):
            t["head"]=[data.text for data in row.find_all('td') if data.text !='']
            
        row_data=[]
        for row in table.find("tbody").findAll('tr'):
            col_data={} if len(t["head"])>3 else []
            for index,data in enumerate(row.find_all('td')):
                text=data.text.split()
                if len(t["head"])>3:
                    if len(text) ==1 and text[0]!='' :
                        col_data[t["head"][index]]=text[0]
                    if len(text) >1:
                        col_data[t["head"][index]]=' '.join(text)
                else:
                    if len(text) ==1 and text[0]!='' :
                        col_data.append(text[0])
                    if len(text) >1:
                        col_data.append(' '.join(text))
            if len(col_data) >1:
                row_data.append(col_data)   
            
        t["body"]=row_data 
        alldata.append(t)  
        
    # print(alldata,index)
    return {
        "NEPSE_INDEX":index,
        "market_status":market_status,
        "market_info":market_info,
        "market_data":alldata
    }   

crawler()


Website Successfully Crawled.
3


{'NEPSE_INDEX': 0,
 'market_status': 'Market Closed',
 'market_info': [{'Share Volume ': ' 8,776,337'},
  {'Turnover ': ' 2,966,899,791'}],
 'market_data': [{'head': ['Market Summary'],
   'body': [['Total Turnover Rs:', '2,966,899,791'],
    ['Total Traded Shares', '8,776,337'],
    ['Total Transactions', '40,401'],
    ['Total Scrips Traded', '218'],
    ['Total Market Capitalization Rs.', '3,623,258.42 Millions'],
    ['Floated Market Capitalization Rs.', '1,290,365.30 Millions']]},
  {'head': ['Index', 'Current', 'Points Change', '%Change'],
   'body': [{'Index': 'NEPSE',
     'Current': '2,587.45',
     'Points Change': '69.51',
     '%Change': '2.62'},
    {'Index': 'Sensitive',
     'Current': '484.93',
     'Points Change': '12.43',
     '%Change': '2.5'},
    {'Index': 'Float',
     'Current': '181.84',
     'Points Change': '5',
     '%Change': '2.68'},
    {'Index': 'Sen. Float',
     'Current': '163.98',
     'Points Change': '4.42',
     '%Change': '2.63'}]},
  {'head': ['